# A function to solve a linear programming problem using OR-Tools library.

Input: 
- solver: A solver object created using pywraplp module of OR-Tools library.
- variable_list: A list of variables used in the problem, created using the NumVar method of the solver object.
- constraint_list: A list of constraints added to the problem, created using the Add method of the solver object.

Output:
- Prints the number of variables and constraints in the problem.
- Solves the problem and prints the optimal solution, time taken, and reduced cost of each variable.
- Prints the dual value and activity level of each constraint.




In [58]:

from ortools.linear_solver import pywraplp

def ShowResults(solver, variable_list, constraint_list):
    """Solve the problem and print the solution."""
    print('# of variables = %d' % solver.NumVariables())
    print('# of constraints = %d' % solver.NumConstraints())

    result_status = solver.Solve()

    # check results are ok
    assert result_status == pywraplp.Solver.OPTIMAL
    assert solver.VerifySolution(1e-7, True)

    print('Problem solved in %f milliseconds' % solver.wall_time())
    print('Optimal objective value = %f' % solver.Objective().Value())
    for variable in variable_list:
        print('%s = %f' % (variable.name(), variable.solution_value()))
    print('Problem solved in %d iterations' % solver.iterations())
    for variable in variable_list:
        print('%s: reduced cost = %f' %
              (variable.name(), variable.reduced_cost()))
    activities = solver.ComputeConstraintActivities()
    for i, constraint in enumerate(constraint_list):
        print(('constraint %d: dual value (shadow price) = %f\n'
               '              final value (activity) = %f\n'
               '              slack  = %f' %
               (i, constraint.dual_value(), activities[constraint.index()],constraint.ub()-activities[constraint.index()])))


In [59]:
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D

def plot_solution(variable_list, constraint_list):
    num_vars = len(variable_list)
    
    if num_vars == 2:
        x_vals = np.linspace(0, 100, 400)
        
        plt.figure(figsize=(10, 6))
        
        for constraint in constraint_list:
            coeffs = constraint.coefficients()
            intercept = constraint.ub()
            slope = -coeffs[0] / coeffs[1] if coeffs[1] != 0 else float('inf')
            if slope != float('inf'):
                intercept /= coeffs[1]
                y_vals = (intercept - coeffs[0] * x_vals / coeffs[1])
                plt.plot(x_vals, y_vals, label=constraint.name())
        
        plt.xlim(0, 50)
        plt.ylim(0, 50)
        
        optimal_x1 = variable_list[0].solution_value()
        optimal_x2 = variable_list[1].solution_value()
        
        plt.plot(optimal_x1, optimal_x2, 'ro', label='Optimal solution')
        
        plt.xlabel(variable_list[0].name())
        plt.ylabel(variable_list[1].name())
        plt.legend()
        plt.title('Feasible Region and Optimal Solution')
        plt.show()
    
    elif num_vars == 3:
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        
        x1_vals = np.linspace(0, 50, 50)
        x2_vals = np.linspace(0, 50, 50)
        x1_vals, x2_vals = np.meshgrid(x1_vals, x2_vals)
        
        for constraint in constraint_list:
            coeffs = constraint.coefficients()
            intercept = constraint.ub()
            if coeffs[2] != 0:
                z_vals = (intercept - coeffs[0] * x1_vals - coeffs[1] * x2_vals) / coeffs[2]
                ax.plot_surface(x1_vals, x2_vals, z_vals, alpha=0.5, label=constraint.name())
        
        optimal_x1 = variable_list[0].solution_value()
        optimal_x2 = variable_list[1].solution_value()
        optimal_x3 = variable_list[2].solution_value()
        
        ax.scatter(optimal_x1, optimal_x2, optimal_x3, color='red', s=100)
        
        ax.set_xlabel(variable_list[0].name())
        ax.set_ylabel(variable_list[1].name())
        ax.set_zlabel(variable_list[2].name())
        plt.title('Feasible Region and Optimal Solution')
        plt.show()


run code

In [60]:

# Define the solver to use
solver = pywraplp.Solver('SolveSimpleSystem', pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)
# Define the problem
infinity = solver.infinity()
x1 = solver.NumVar(0.0, 200, 'X1')
x2 = solver.NumVar(0.0, 1000, 'X2')
x3 = solver.NumVar(0.0, 100, 'X3')

solver.Maximize(350 * x1 + 90 * x2 + 900 * x3)
c0 = solver.Add(5000 * x1 + 1000 * x2 + 10000 * x3  == 1000000, 'capital')
c1 = solver.Add(10000 * x3 <= 250000, 'risk')
c2 = solver.Add(5000 * x1 >= 400000, 'safety')

ShowResults(solver, [x1, x2, x3], [c0, c1, c2])

solver.Solve()
print("Rafinery Solution:")
print("Rafinery A work days:", xa.solution_value())
print("Rafinery B work days:", xb.solution_value())
print("Cost : ", solver.Objective().Value())

#plot_solution([x1, x2], [c0, c1, c2])


# of variables = 3
# of constraints = 3
Problem solved in 0.000000 milliseconds
Optimal objective value = 82000.000000
X1 = 80.000000
X2 = 600.000000
X3 = 0.000000
Problem solved in 0 iterations
X1: reduced cost = 0.000000
X2: reduced cost = 0.000000
X3: reduced cost = 0.000000
constraint 0: dual value (shadow price) = 0.090000
              final value (activity) = 1000000.000000
              slack  = 0.000000
constraint 1: dual value (shadow price) = 0.000000
              final value (activity) = 0.000000
              slack  = 250000.000000
constraint 2: dual value (shadow price) = -0.020000
              final value (activity) = 400000.000000
              slack  = inf


In [61]:
# Define the solver to use
solver = pywraplp.Solver.CreateSolver('GLOP')

# Define the problem
infinity = solver.infinity()
x1 = solver.NumVar(0.0, infinity, 'XA')
x2 = solver.NumVar(0.0, infinity, 'XB')

solver.Maximize(40 * x1 + 30 * x2)
c0 = solver.Add(x1 + x2 <= 12, 'Constraint 1')
c1 = solver.Add(2 * x1 + x2 <= 16, 'Constraint 2')


ShowResults(solver, [x1, x2], [c0, c1])

plot_solution([x1, x2], [c0, c1])

# of variables = 2
# of constraints = 2
Problem solved in 0.000000 milliseconds
Optimal objective value = 400.000000
XA = 4.000000
XB = 8.000000
Problem solved in 2 iterations
XA: reduced cost = -0.000000
XB: reduced cost = -0.000000
constraint 0: dual value (shadow price) = 20.000000
              final value (activity) = 12.000000
              slack  = 0.000000
constraint 1: dual value (shadow price) = 10.000000
              final value (activity) = 16.000000
              slack  = 0.000000


AttributeError: 'Constraint' object has no attribute 'coefficients'

<Figure size 1000x600 with 0 Axes>